In [42]:
import zhipuai

In [43]:
from utils import get_llm_classification, trim_string, get_similarity_column_name

In [77]:
def get_llm_sql(question, years, columns):
    assistant_role_msg = """你的任务是将我所提出的问题转换为SQL。我会给你提供要查询的表名、涉及到的年份和涉及到的数据库表列名，你需要基于此写出对应的SQL语句。
    接下来我将给你提供几个例子：
    Input: 查询的表名为company，涉及到的年份：2020、2021，涉及到的列名有: 法定代表人,公司名称,股票简称,股票代码,合同资产,报告年份。问题：2021年公司名称为晋西车轴股份有限公司的公司法定代表人与2020年相比是否都是相同的？
    Output: select 公司名称,股票简称,报告年份,法定代表人 from company where 报告年份 in ('2021', '2020') and (公司名称 in ('晋西车轴股份有限公司') or 股票简称 in ('晋西车轴股份有限公司'));
    Input: 查询的表名为company，涉及到的年份：2021，涉及到的列名有: 公司名称,硕士人员,职工总数,股票代码,股票简称,博士人员,报告年份。问题：朗新科技2021年的硕士员工人数有多少？
    Output: select 公司名称,股票简称,报告年份,硕士人员 from company where 报告年份 in ('2021') and (公司名称 in ('朗新科技') or 股票简称 in ('朗新科技');
    Input:查询的表名为company，涉及到的年份：2019，涉及到的列名有: 公司名称,固定资产,无形资产,股票代码,股票简称,报告年份。问题：2019年银星能源固定资产和无形资产分别是多少元?
    Output: select 公司名称,股票简称,报告年份,固定资产,无形资产 from company where 报告年份 in ('2019') and (公司名称 in ('银星能源') or 股票简称 in ('银星能源');
    Input: 查询的表名为company，涉及到的年份：2019，涉及到的列名有: 公司名称,股票代码,股票简称,报告年份,注册地址,资产总计。问题：在北京注册的上市公司中，2019年资产总额最高的前四家上市公司是哪些家？金额为？
    Output: select 公司名称,股票简称,报告年份,资产总计 from company where 报告年份 in ('2019') and 注册地址 like '%北京%' order by 资产总计 desc limit 4;
    ……
    对于Output应该严格按照sqlite的sql格式输出，并且所有select的字段中都要包含“公司名称,股票简称,报告年份”这几项，还要注意区分WHERE筛选条件中到底是用“公司名称”还是“股票简称”，不要输出任何多余的字符。
    """
    columns = ["公司名称", "股票简称", "股票代码", "报告年份"] + columns
    columns_msg = ','.join(columns).replace(" ", '')
    years_msg = '、'.join([str(i) + "年" for i in years]).replace(" ", '')
    message =f"查询的表名为company，涉及到的年份：{years_msg}，涉及到的列名有: {columns_msg}。问题：{question}"
    response = zhipuai.model_api.invoke(
        model="chatglm_pro",
        prompt=[
            {"role": "user", "content": assistant_role_msg + f"\n那么，再给定Input: {message}，应该Output:"},
        ],
        temperature=1.0,
        ref={"enable": False}
    )

    result = response["data"]["choices"][0]["content"].replace('，', ',')
    return trim_string(result)


In [82]:
q = "津药药业2021年和2019年的法定代表人与上一年是否相同?"
d = get_llm_classification(q)
cs = [col for keyword in d["keyword"] for col in get_similarity_column_name(keyword)]
sql = get_llm_sql(q, d["year"], cs)


Content: 法定代表人, Score: 1.1384489184695923e-10
Content: 公司名称, Score: 265.4627685546875


In [83]:
sql

"select 公司名称,股票简称,报告年份,法定代表人 from company where 报告年份 in ('2019', '2021') and (公司名称 in ('津药药业') or 股票简称 in ('津药药业'));"

In [51]:
import json
import sqlite3


In [48]:
# 创建连接
conn = sqlite3.connect("../prepare_data/company.db")
cursor = conn.cursor()


In [84]:
# 查询数据
cursor.execute(sql)
results = cursor.fetchall()
# 获取列名
columns = [desc[0] for desc in cursor.description]
# 转换查询结果为字典
data = [dict(zip(columns, row)) for row in results]
# 转换字典为JSON格式
json_data = json.dumps(data, ensure_ascii=False, indent=4)

In [85]:
json_data


'[\n    {\n        "公司名称": "津药药业股份有限公司",\n        "股票简称": "津药药业",\n        "报告年份": 2021,\n        "法定代表人": "刘欣"\n    },\n    {\n        "公司名称": "津药药业股份有限公司",\n        "股票简称": "津药药业",\n        "报告年份": 2019,\n        "法定代表人": "张杰"\n    }\n]'

In [86]:
def answer_normalize(question, answer):
    assistant_msg = f"""请你根据查询结果回答问题，要求语言流畅，表意清晰，完整通顺。
    查询结果：{answer}
    问题：{question}
    回答：
    """
    response = zhipuai.model_api.invoke(
        model="chatglm_pro",
        prompt=[
            {"role": "user", "content": assistant_msg},
        ],
        temperature=1.0,
        ref={"enable": False}
    )

    result = response["data"]["choices"][0]["content"].replace('，', ',')
    return trim_string(result)


In [87]:
answer_normalize(q, json_data)


'津药药业 2021 年的法定代表人是刘欣,2019 年的法定代表人是张杰。因此,2021 年和 2019 年的法定代表人并不相同。'